Plan:
1. get FEOBV HC vs Smokers difference map in MNI152 space
2. Parcellate the difference map using a DK atlas in MNI152 space
3. Select only left hemisphere of the parcellation

4. get gene expression scale033 data from Justine's paper
5. Convert expression data to DK atlas

6. Correlate FEOBV parcellation with gene expression using spin tests